<h1>Asan Dopollos Model</h1>

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, GradientBoostingClassifier, RandomForestRegressor, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC, SVR
from sklearn.neural_network import MLPClassifier
from sklearn import metrics
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


# Import datasets
network = pd.read_csv('network.csv')
inspections = pd.read_csv('inspections.csv')
pipes = pd.read_csv('sample_submission.csv')

## Data Visualization

In [2]:
network.head()

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,489616,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,63.0,7.792,0.4,0,0,False
1,489645,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,2.080,0.4,0,0,False
2,489646,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,111.173,0.4,3,0,False
3,519950,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,200.0,14.304,0.4,0,0,False
4,1332439,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,160.0,22.669,0.4,1,0,False


In [3]:
pipes.head()

,PipeId,Incidence
0,446859944,0
1,428124500,0
2,438428871,0
3,429034569,0
4,411184477,0


In [4]:
inspections.head()

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0
1,188889914,ZRV-00001406,2010,2010-10-01,22,4,0
2,188889907,ZRV-00001406,2010,2010-10-01,22,4,0
3,188673355,ZRV-00001406,2010,2010-10-01,22,4,0
4,188558922,ZRV-00001406,2010,2010-10-01,22,4,0


Merging the df with an inner join using PipeId

In [5]:
# Merge both dataframes
df = inspections.merge(
                        network,
                        how = 'inner',
                        on=['PipeId'] 
                        )

df

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
1,189311802,ZRV-00001972,2012,2012-10-01,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
2,189311802,ZRV-00001972,2014,2014-10-08,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
3,189311802,ZRV-00001972,2016,2016-10-14,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
4,189311802,ZRV-00001972,2018,2018-10-09,24,4,0,Barcelona,Sentmenat,2001,PE,Gas natural,160.0,117.831,4.000,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,ZRV-00004581,2020,2020-12-31,24,4,0,Valencia,Betera,1993,PE,Gas natural,63.0,1.237,4.000,0,0,False
6345340,56907425,ZRV-00004581,2020,2020-12-31,24,4,0,Valencia,Betera,2000,PE,Gas natural,160.0,169.732,4.000,1,0,False
6345341,7843615,ZRV-00002112,2021,2020-12-31,24,4,0,Barcelona,Sabadell,1995,FD,Gas natural,150.0,0.361,0.025,0,0,True
6345342,56922456,ZRV-00004581,2020,2020-12-31,24,4,0,Valencia,Betera,1993,PE,Gas natural,63.0,23.306,4.000,0,0,False


| Field               | Meaning                                              |
|---------------------|------------------------------------------------------|
| PipeId              | Unique identifier for the pipe                       |
| Province            | Spanish province where the pipe is located           |
| Town                | Town or city where the pipe is located               |
| YearBuilt           | Year in which the pipe was built and installed       |
| Material            | Material in which the pipe is built                  | 
| GasType             | Type of gas that runs through the pipe               |
| Diameter            | Diameter of the pipe (in mm)                         |
| Length              | Length of the pipe (in m)                            |
| Pressure            | Pressure of the gas that runs through the pipe (bar) |
| NumConnections      | Number of connections (external)                     |
| NumConnectionsUnder | Number of connections (internal and buried)          |
| BoolBridle          | Whether the pipe is bridled (True) or welded (False) |



## Data preparation

Dates will be converted to datetime.

In [6]:
df['InspectionDate'] = pd.to_datetime(df['InspectionDate'])

Column Dop: MaintenaceId, Town, NumConnectionsUnder

In [7]:
df = df.drop(['MaintenanceId','NumConnectionsUnder','Town'],axis=1)
df

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle
0,189311802,2010,2010-10-01,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
1,189311802,2012,2012-10-01,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
2,189311802,2014,2014-10-08,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
3,189311802,2016,2016-10-14,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
4,189311802,2018,2018-10-09,24,4,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,2020,2020-12-31,24,4,0,Valencia,1993,PE,Gas natural,63.0,1.237,4.000,0,False
6345340,56907425,2020,2020-12-31,24,4,0,Valencia,2000,PE,Gas natural,160.0,169.732,4.000,1,False
6345341,7843615,2021,2020-12-31,24,4,0,Barcelona,1995,FD,Gas natural,150.0,0.361,0.025,0,True
6345342,56922456,2020,2020-12-31,24,4,0,Valencia,1993,PE,Gas natural,63.0,23.306,4.000,0,False


Severity values will be inverted, meaning that they will go from 0-3 going from no incidence to highest severity

In [8]:
inverse_sev = {4:0,3:1,2:2,1:3}
list = []
for sev in df["Severity"]:
    list.append(inverse_sev[sev]) 

df['Severity'] = list
df

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle
0,189311802,2010,2010-10-01,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
1,189311802,2012,2012-10-01,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
2,189311802,2014,2014-10-08,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
3,189311802,2016,2016-10-14,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
4,189311802,2018,2018-10-09,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,2020,2020-12-31,24,0,0,Valencia,1993,PE,Gas natural,63.0,1.237,4.000,0,False
6345340,56907425,2020,2020-12-31,24,0,0,Valencia,2000,PE,Gas natural,160.0,169.732,4.000,1,False
6345341,7843615,2021,2020-12-31,24,0,0,Barcelona,1995,FD,Gas natural,150.0,0.361,0.025,0,True
6345342,56922456,2020,2020-12-31,24,0,0,Valencia,1993,PE,Gas natural,63.0,23.306,4.000,0,False


## Feature Engineering

### Material properties
Table with the material type:

|Material Type | Corrodes | Young's Module (GPa) |
|--------------|----------|----------------------|
|AO: aluminum oxide     | No     |  380 |
|CU: copper     | Yes     | 120 |
|FD: ductil function     | No     |  172 |
|FG: fiber glass     | No     | 50 |
|FI: flame inhibitant     | No     | 4 |
|FO: unkown     | No     | 4 |
|PA: polyamid     | No     | 6 |
|PE: polyehtylene     | No     | 1 | 
|PN: polyethylene naphthalate     | No     | 4.5 |
|PV: polyvinyl      | No     | 4 | 
|ZD: zinc diecasting     | No     | 100 |

In [9]:
# Adding corrosive properties to the elements
df ['Corrosive'] = 0
indices = df[df.Material == 'CU'].index
df['Corrosive'][indices] = 1
# Adding Youngs Modulus to see which ones are more prone to cracks
youngs_dict = {'AO':380,'CU':120,'FD':172,'FG':50,'FI':4,'FO':4,'PA':6,'PE':1,'PN':4.5,'PV':4,'ZD':100}
lst = []
for idx in df.index:
    lst.append(youngs_dict[df['Material'][idx]])

df['Youngs_Mod'] = lst
df

C:\Users\rodri\AppData\Local\Temp\ipykernel_16348\4004144257.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Corrosive'][indices] = 1


,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod
0,189311802,2010,2010-10-01,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False,0,1.0
1,189311802,2012,2012-10-01,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False,0,1.0
2,189311802,2014,2014-10-08,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False,0,1.0
3,189311802,2016,2016-10-14,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False,0,1.0
4,189311802,2018,2018-10-09,24,0,0,Barcelona,2001,PE,Gas natural,160.0,117.831,4.000,0,False,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,2020,2020-12-31,24,0,0,Valencia,1993,PE,Gas natural,63.0,1.237,4.000,0,False,0,1.0
6345340,56907425,2020,2020-12-31,24,0,0,Valencia,2000,PE,Gas natural,160.0,169.732,4.000,1,False,0,1.0
6345341,7843615,2021,2020-12-31,24,0,0,Barcelona,1995,FD,Gas natural,150.0,0.361,0.025,0,True,0,172.0
6345342,56922456,2020,2020-12-31,24,0,0,Valencia,1993,PE,Gas natural,63.0,23.306,4.000,0,False,0,1.0


Normalization

In [10]:
for column in ['Diameter', 'Length', 'Pressure', 'Youngs_Mod']:
   df[column] = (df[column] - df[column].mean()) / df[column].std()

### Age of the pipe

In [11]:
df['Age'] = df["InspectionYear"] - df["YearBuilt"]

### Previous revision severity

In [12]:
df = df.sort_values(['PipeId','InspectionDate'])
df = df.reset_index(drop=True)

prev_sev = 0
previous_severity = []

for ind in df.index:
    previous_severity.append(prev_sev)
    prev_sev = df['Severity'][ind]

df['PrevSeverity'] = previous_severity
df[df.groupby('PipeId').cumcount() == 0]['PrevSeverity'] = 0

df

C:\Users\rodri\AppData\Local\Temp\ipykernel_16348\221099879.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[df.groupby('PipeId').cumcount() == 0]['PrevSeverity'] = 0


,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,Age,PrevSeverity
0,489616,2013,2013-05-08,16,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,1,0
1,489616,2015,2015-03-04,22,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,3,0
2,489616,2016,2017-01-12,22,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,4,0
3,489616,2018,2018-12-19,23,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,6,0
4,489616,2020,2020-12-05,24,0,0,Ciudad Real,2012,PE,Gas natural,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,451995260,2019,2019-10-23,24,0,0,Lleida,2003,PE,Gas natural,-0.516448,0.426301,-0.419375,2,False,0,-0.35414,16,0
6345340,451995309,2013,2013-10-07,24,0,0,Lleida,1998,PE,Gas natural,1.406152,0.771957,-0.419375,0,False,0,-0.35414,15,0
6345341,451995309,2015,2015-10-05,24,0,0,Lleida,1998,PE,Gas natural,1.406152,0.771957,-0.419375,0,False,0,-0.35414,17,0
6345342,451995309,2017,2017-10-02,24,0,0,Lleida,1998,PE,Gas natural,1.406152,0.771957,-0.419375,0,False,0,-0.35414,19,0


### Including the Earthquakes Data:
The accumulated number of seisms between revisions will be used

In [13]:
df_seism = pd.read_csv('Sismic_activity_dataframe.csv')
# Seism dates to datetimes
df_seism['Fecha'] = pd.to_datetime(df_seism['Fecha'])

df = pd.merge_asof(df.sort_values(by='InspectionDate'),df_seism.sort_values(by='Fecha')
            ,left_on=['InspectionDate'],right_on=['Fecha'],by='Province')

# Dropping the extra variables
df = df.drop(['Fecha','Year','Seism'],axis=1)
df = df.sort_values(['PipeId','InspectionDate'])
df = df.fillna(method='ffill')
df = df.sort_values(['PipeId','InspectionDate'])
df = df.reset_index(drop=True)


In [14]:
diff = df['AcumSeism'].diff()
diff

0             NaN
1            34.0
2            43.0
3            20.0
4            47.0
            ...  
6345339     667.0
6345340   -1608.0
6345341     267.0
6345342     674.0
6345343     670.0
Name: AcumSeism, Length: 6345344, dtype: float64

### Accumulated number of incidence per pipe and accumulated number of revisions

In [98]:
accum_incidence_list = [0]
incidence_count = df['Incidence'][0]
revision_count = 1
revision_list = [0]

for ind in range(1, len(df.index)):
    if df['PipeId'][ind] != df['PipeId'][ind-1]:
        incidence_count = 0
        revision_count = 0
    accum_incidence_list.append(incidence_count)
    revision_list.append(revision_count)
    incidence_count = incidence_count + df['Incidence'][ind]
    revision_count += 1

df['AcumIncidence'] = accum_incidence_list
df['AcumRevisions'] = revision_list

In [99]:
df

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,Province,YearBuilt,Material,GasType,...,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,Age,PrevSeverity,AcumSeism,AcumIncidence,AcumRevisions
0,489616,2013,2013-05-08,16,0,0,Ciudad Real,2012,PE,Gas natural,...,-0.381338,0,False,0,-0.35414,1,0,62.0,0,0
1,489616,2015,2015-03-04,22,0,0,Ciudad Real,2012,PE,Gas natural,...,-0.381338,0,False,0,-0.35414,3,0,96.0,0,1
2,489616,2016,2017-01-12,22,0,0,Ciudad Real,2012,PE,Gas natural,...,-0.381338,0,False,0,-0.35414,4,0,139.0,0,2
3,489616,2018,2018-12-19,23,0,0,Ciudad Real,2012,PE,Gas natural,...,-0.381338,0,False,0,-0.35414,6,0,159.0,0,3
4,489616,2020,2020-12-05,24,0,0,Ciudad Real,2012,PE,Gas natural,...,-0.381338,0,False,0,-0.35414,8,0,206.0,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,451995260,2019,2019-10-23,24,0,0,Lleida,2003,PE,Gas natural,...,-0.419375,2,False,0,-0.35414,16,0,1991.0,0,3
6345340,451995309,2013,2013-10-07,24,0,0,Lleida,1998,PE,Gas natural,...,-0.419375,0,False,0,-0.35414,15,0,383.0,0,0
6345341,451995309,2015,2015-10-05,24,0,0,Lleida,1998,PE,Gas natural,...,-0.419375,0,False,0,-0.35414,17,0,650.0,0,1
6345342,451995309,2017,2017-10-02,24,0,0,Lleida,1998,PE,Gas natural,...,-0.419375,0,False,0,-0.35414,19,0,1324.0,0,2


One hot encoding the categorical values with pandas get dummies

In [100]:
# Drop additional variables
df = pd.get_dummies(df)
df.head(4)

,PipeId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,YearBuilt,Diameter,Length,Pressure,...,Material_FG,Material_FI,Material_FO,Material_PA,Material_PE,Material_PN,Material_PV,Material_ZD,GasType_Gas natural,GasType_Gas propano
0,489616,2013,2013-05-08,16,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0
1,489616,2015,2015-03-04,22,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0
2,489616,2016,2017-01-12,22,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0
3,489616,2018,2018-12-19,23,0,0,2012,-0.988359,-0.398361,-0.381338,...,0,0,0,0,1,0,0,0,1,0


The dataframe will be reindexed by means of the PipeId and Date

In [101]:
df = df.set_index(['PipeId', 'InspectionDate','Severity'])
df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 6345344 entries, (489616, Timestamp('2013-05-08 00:00:00'), 0) to (451995309, Timestamp('2019-10-26 00:00:00'), 0)
Data columns (total 67 columns):
 #   Column                Dtype  
---  ------                -----  
 0   InspectionYear        int64  
 1   MonthsLastRev         int64  
 2   Incidence             int64  
 3   YearBuilt             int64  
 4   Diameter              float64
 5   Length                float64
 6   Pressure              float64
 7   NumConnections        int64  
 8   BoolBridle            bool   
 9   Corrosive             int64  
 10  Youngs_Mod            float64
 11  Age                   int64  
 12  PrevSeverity          int64  
 13  AcumSeism             float64
 14  AcumIncidence         int64  
 15  AcumRevisions         int64  
 16  Province_Albacete     uint8  
 17  Province_Alicante     uint8  
 18  Province_Almeria      uint8  
 19  Province_Avila        uint8  
 20  Province_Barcelona    uint8

Splitting in a train-test set

In [102]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Incidence'],axis=1),df['Incidence'])

Balancing the dataframe as it is quite unbalanced

In [103]:
X_train['Incidence'] = y_train

In [104]:
# First we check how balanced is the dataset
Incidence = X_train[X_train.Incidence == 1]
not_Incidence = X_train[X_train.Incidence == 0]

# index property contains the indexes of the not_Incidence dataset
not_Incidence_indexes = not_Incidence.index

# get as many not_incidence indexes (randomly) as incidence occurrences
np.random.seed(42)
random_not_Incidence_indexes = np.random.choice(not_Incidence_indexes, len(Incidence))

# Keep those entries in not_Incidence
not_Incidence = X_train.loc[random_not_Incidence_indexes]

# We will form the balanced dataset concatenating Incidence and not_Incidence
balanced_df = Incidence.append(not_Incidence)
balanced_df.sort_index()
X_train = balanced_df

y_train = X_train['Incidence']
X_train = X_train.drop(['Incidence'],axis=1)

C:\Users\rodri\AppData\Local\Temp\ipykernel_21828\3047329077.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_df = Incidence.append(not_Incidence)


## Training and testing the model

Neural Network

In [105]:
# forecaster_nn = MLPClassifier()
# forecaster_nn.fit(X_train, y_train)
# Incidence_pred = forecaster_nn.predict(X_test)
# print(metrics.roc_auc_score(y_test,Incidence_pred))

Random Forest classifier

In [106]:
forecaster_rf = RandomForestClassifier()
# forecaster_rf.fit(X_train, y_train)
# Incidence_pred = forecaster_rf.predict(X_test)
# print(metrics.roc_auc_score(y_test,Incidence_pred))

Gradient boosting classifier

In [107]:
# forecaster_gb = GradientBoostingClassifier()
# forecaster_gb.fit(X_train, y_train)
# Incidence_pred = forecaster_gb.predict(X_test)
# print(metrics.roc_auc_score(y_test,Incidence_pred))

### Hyperparameter tuning

#### Random forest classifier

In [108]:
# Define parameters to test
# grid_param_rf = {
#     'n_estimators' : range(115, 125, 5),
#     'criterion' : ['gini', 'entropy', 'log_loss'], 
#     'max_depth' : range(10, 30, 5),
#     'min_samples_leaf' : [1],
#     'min_samples_split' : range(2,5,1),
#     'max_features' : ['sqrt', 'log2', 'auto']
# }

In [109]:
# grid_search_rf = GridSearchCV(estimator=forecaster_rf, param_grid=grid_param_rf, scoring='roc_auc', cv=5, n_jobs=-1, verbose=3)
# grid_search_rf.fit(X_train, y_train)

In [110]:
# Get the best parameters from the cross validation test
# grid_search_rf.best_params_

Re-test the model with the new parameters

In [111]:
forecaster_rf = RandomForestClassifier(criterion= 'entropy',
                                       max_depth= 20,
                                       max_features= 'log2',
                                       min_samples_leaf= 1,
                                       min_samples_split= 4,
                                       n_estimators= 115)
forecaster_rf.fit(X_train, y_train)
Incidence_pred = forecaster_rf.predict(X_test)
print(metrics.roc_auc_score(y_test,Incidence_pred))

0.8434329104185251


#### Gradient boosting classifier

In [112]:
# Define parameters to test
# grid_param_gb = {
#     'learning_rate' : [0.01, 0.1, 1],
#     'n_estimators' : range(50, 70, 10),
#     'criterion' : ['friedman_mse', 'squared_error'], 
#     'max_depth' : range(1, 7, 2),
#     'min_samples_leaf' : [1, 2, 3],
#     'min_samples_split' : [2, 3, 4, 5],
#     'max_features' : ['sqrt', 'log2']
# }

In [113]:
# grid_search_gb = GridSearchCV(estimator=forecaster_gb, param_grid=grid_param_gb, scoring='roc_auc', cv=6, n_jobs=-1, verbose=3)
# grid_search_gb.fit(X_train, y_train)

In [114]:
# Get the best parameters from the cross validation test
# grid_search_gb.best_params_

Re-test the model with the new parameters

In [115]:
# forecaster_gb = GradientBoostingClassifier(learning_rate= 0.1,
#                                            n_estimators= 60,
#                                            criterion= 'friedman_mse',
#                                            max_depth= 5,
#                                            min_samples_leaf= 1,
#                                            min_samples_split= 3,
#                                            max_features= 'sqrt')
# forecaster_gb.fit(X_train, y_train)
# Incidence_pred = forecaster_gb.predict(X_test)
# print(metrics.roc_auc_score(y_test,Incidence_pred))

#### Neural Network

In [116]:
# Define parameters to test
# grid_param_nn = {
#     'activation' : ['identity', 'logistic', 'tanh', 'relu'],
#     'solver' : ['lbfgs', 'sgd', 'adam'], 
#     'learning_rate' : ['constant', 'invscaling', 'adaptive']
# }

In [117]:
# grid_search_nn = GridSearchCV(estimator=forecaster_nn, param_grid=grid_param_nn, scoring='roc_auc', cv=5, n_jobs=-1, verbose=3)
# grid_search_nn.fit(X_train, y_train)

In [118]:
# Get the best parameters from the cross validation test
# grid_search_nn.best_params_

Re-test the model with the new parameters

In [119]:
# forecaster_nn = MLPClassifier(activation='relu',
#                               learning_rate='invscaling',
#                               solver='adam')

# forecaster_nn.fit(X_train, y_train)
# Incidence_pred = forecaster_nn.predict(X_test)
# print(metrics.roc_auc_score(y_test,Incidence_pred))

## Creating prediction dataset

Merging with the PipeId from the Submission dataset

In [120]:
pipes = pipes.drop(['Incidence'],axis=1)
pred_X = network.merge(
                pipes,
                how='inner',
                on=['PipeId']
)
pred_X

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,489780,Toledo,Añover de Tajo,2009,PE,Gas natural,110.0,7.485,5.0,0,0,False
1,489790,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,4.436,4.0,0,0,False
2,489792,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,11.769,4.0,1,0,False
3,489793,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,6.172,4.0,1,0,False
4,489981,Toledo,Añover de Tajo,2012,PE,Gas natural,110.0,49.031,4.0,0,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
909728,416223449,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.460,45.0,0,0,False
909729,416223471,La Rioja,Cirueña,2011,AO,Gas natural,50.8,0.230,45.0,0,0,False
909730,416425646,La Rioja,Nájera,2011,AO,Gas natural,254.0,1.999,45.0,0,0,False
909731,416425691,La Rioja,Nájera,2011,AO,Gas natural,254.0,3.000,45.0,0,0,False


Drop columns

In [121]:
pred_X = pred_X.drop(['Town', 'NumConnectionsUnder'],axis=1)
pred_X

,PipeId,Province,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,BoolBridle
0,489780,Toledo,2009,PE,Gas natural,110.0,7.485,5.0,0,False
1,489790,Toledo,2012,PE,Gas natural,110.0,4.436,4.0,0,False
2,489792,Toledo,2012,PE,Gas natural,110.0,11.769,4.0,1,False
3,489793,Toledo,2012,PE,Gas natural,110.0,6.172,4.0,1,False
4,489981,Toledo,2012,PE,Gas natural,110.0,49.031,4.0,0,False
...,...,...,...,...,...,...,...,...,...,...
909728,416223449,La Rioja,2011,AO,Gas natural,50.8,0.460,45.0,0,False
909729,416223471,La Rioja,2011,AO,Gas natural,50.8,0.230,45.0,0,False
909730,416425646,La Rioja,2011,AO,Gas natural,254.0,1.999,45.0,0,False
909731,416425691,La Rioja,2011,AO,Gas natural,254.0,3.000,45.0,0,False


Material Type

In [122]:
pred_X['Corrosive'] = 0
indices = pred_X[pred_X.Material == 'CU'].index
pred_X['Corrosive'][indices] = 1
# Adding Youngs Modulus to see which ones are more prone to cracks
youngs_dict = {'AO':380,'CU':120,'FD':172,'FG':50,'FI':4,'FO':4,'PA':6,'PE':1,'PN':4.5,'PV':4,'ZD':100}
lst = []
for idx in pred_X.index:
    lst.append(youngs_dict[pred_X['Material'][idx]])

pred_X['Youngs_Mod'] = lst

C:\Users\rodri\AppData\Local\Temp\ipykernel_21828\2854941000.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_X['Corrosive'][indices] = 1


Normalization

In [123]:
for column in ['Diameter', 'Length', 'Pressure','Youngs_Mod']:
   pred_X[column] = (pred_X[column] - pred_X[column].mean()) / pred_X[column].std()

Add Previous severity variable

In [124]:
last_rev_df = df
last_rev_df.reset_index(inplace=True)
last_rev_df = last_rev_df.drop_duplicates(subset=['PipeId'], keep='last')
pred_X = pred_X.merge(last_rev_df[['PipeId', 'Severity', 'AcumRevisions', 'AcumIncidence','Incidence']], on = 'PipeId', how = 'left')
pred_X.rename(columns = {'Severity':'PrevSeverity'}, inplace = True)

Changing the NaN in Severity to 0, as some of them have never been revised. Same will happen with the accumulated values

In [125]:
pred_X['PrevSeverity'] = pred_X['PrevSeverity'].replace(np.nan, 0)
pred_X['AcumRevisions'] = pred_X['AcumRevisions'].replace(np.nan, 0)
pred_X['AcumIncidence'] = pred_X['AcumIncidence'].replace(np.nan, 0)
pred_X['Incidence'] = pred_X['Incidence'].replace(np.nan, 0)

pred_X['PrevSeverity'] = pred_X['PrevSeverity'].astype('int')
pred_X['AcumIncidence'] = pred_X['AcumIncidence'].astype('int')
pred_X['AcumRevisions'] = pred_X['AcumRevisions'].astype('int')
pred_X['Incidence'] = pred_X['Incidence'].astype('int')

Accumulated Incidences

In [126]:
pred_X['AcumIncidence'] = pred_X['AcumIncidence'] + pred_X['Incidence']
pred_X = pred_X.drop(['Incidence'],axis=1)

Accumulated Revisions

In [127]:
pred_X['AcumRevisions'] = pred_X['AcumRevisions'] + 1

Encoding and keeping the provinces in a list to later integrate the earthquake information

In [128]:
provinces = pred_X[['Province','PipeId']]
pred_X = pd.get_dummies(pred_X)
pred_X

,PipeId,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,...,Material_CU,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano
0,489780,2009,-0.111998,-0.421161,0.387956,0,False,0,-0.350055,0,...,0,0,0,0,0,1,0,0,1,0
1,489790,2012,-0.111998,-0.462561,0.216777,0,False,0,-0.350055,0,...,0,0,0,0,0,1,0,0,1,0
2,489792,2012,-0.111998,-0.362993,0.216777,1,False,0,-0.350055,0,...,0,0,0,0,0,1,0,0,1,0
3,489793,2012,-0.111998,-0.438989,0.216777,1,False,0,-0.350055,0,...,0,0,0,0,0,1,0,0,1,0
4,489981,2012,-0.111998,0.142952,0.216777,0,False,0,-0.350055,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,416223449,2011,-1.138929,-0.516547,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,0,1,0
909729,416223471,2011,-1.138929,-0.519670,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,0,1,0
909730,416425646,2011,2.385941,-0.495651,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,0,1,0
909731,416425691,2011,2.385941,-0.482059,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,0,1,0


Obtaining the MonthLastRev column will be done through a prediction

prepair training dataset

In [129]:
df_regressor = df
df_regressor = df_regressor.drop(["Material_FO", "Material_PV", "Incidence", "InspectionDate", "Severity", "InspectionYear", "Age",'AcumSeism'], axis = 1)

df_regressor = df_regressor.set_index(['PipeId'])
df_regressor

,MonthsLastRev,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,...,Material_CU,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,,,,,,,,,,,,,,,,,,,,,
489616,16,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0
489616,22,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0
489616,22,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0
489616,23,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0
489616,24,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451995260,24,2003,-0.516448,0.426301,-0.419375,2,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0
451995309,24,1998,1.406152,0.771957,-0.419375,0,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0
451995309,24,1998,1.406152,0.771957,-0.419375,0,False,0,-0.35414,0,...,0,0,0,0,0,1,0,0,1,0


In [130]:
df_regressor_y = df_regressor["MonthsLastRev"]
df_regressor = df_regressor.drop(['MonthsLastRev'], axis = 1)

set pipe id in the index for the predictor

In [131]:
pred_X = pred_X.set_index(['PipeId'])
pred_X

,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,AcumRevisions,...,Material_CU,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,,,,,,,,,,,,,,,,,,,,,
489780,2009,-0.111998,-0.421161,0.387956,0,False,0,-0.350055,0,5,...,0,0,0,0,0,1,0,0,1,0
489790,2012,-0.111998,-0.462561,0.216777,0,False,0,-0.350055,0,5,...,0,0,0,0,0,1,0,0,1,0
489792,2012,-0.111998,-0.362993,0.216777,1,False,0,-0.350055,0,5,...,0,0,0,0,0,1,0,0,1,0
489793,2012,-0.111998,-0.438989,0.216777,1,False,0,-0.350055,0,5,...,0,0,0,0,0,1,0,0,1,0
489981,2012,-0.111998,0.142952,0.216777,0,False,0,-0.350055,0,5,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416223449,2011,-1.138929,-0.516547,7.235145,0,False,0,3.210377,0,1,...,0,0,0,0,0,0,0,0,1,0
416223471,2011,-1.138929,-0.519670,7.235145,0,False,0,3.210377,0,1,...,0,0,0,0,0,0,0,0,1,0
416425646,2011,2.385941,-0.495651,7.235145,0,False,0,3.210377,0,2,...,0,0,0,0,0,0,0,0,1,0


order columns

In [132]:
pred_X = pred_X[df_regressor.columns]
pred_X

,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,AcumIncidence,...,Material_CU,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,,,,,,,,,,,,,,,,,,,,,
489780,2009,-0.111998,-0.421161,0.387956,0,False,0,-0.350055,0,0,...,0,0,0,0,0,1,0,0,1,0
489790,2012,-0.111998,-0.462561,0.216777,0,False,0,-0.350055,0,0,...,0,0,0,0,0,1,0,0,1,0
489792,2012,-0.111998,-0.362993,0.216777,1,False,0,-0.350055,0,0,...,0,0,0,0,0,1,0,0,1,0
489793,2012,-0.111998,-0.438989,0.216777,1,False,0,-0.350055,0,0,...,0,0,0,0,0,1,0,0,1,0
489981,2012,-0.111998,0.142952,0.216777,0,False,0,-0.350055,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416223449,2011,-1.138929,-0.516547,7.235145,0,False,0,3.210377,0,0,...,0,0,0,0,0,0,0,0,1,0
416223471,2011,-1.138929,-0.519670,7.235145,0,False,0,3.210377,0,0,...,0,0,0,0,0,0,0,0,1,0
416425646,2011,2.385941,-0.495651,7.235145,0,False,0,3.210377,0,0,...,0,0,0,0,0,0,0,0,1,0


In [133]:
df_regressor

,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,AcumIncidence,...,Material_CU,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,,,,,,,,,,,,,,,,,,,,,
489616,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0
489616,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0
489616,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0
489616,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0
489616,2012,-0.988359,-0.398361,-0.381338,0,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451995260,2003,-0.516448,0.426301,-0.419375,2,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0
451995309,1998,1.406152,0.771957,-0.419375,0,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0
451995309,1998,1.406152,0.771957,-0.419375,0,False,0,-0.35414,0,0,...,0,0,0,0,0,1,0,0,1,0


Predict months

In [134]:
regressor = GradientBoostingRegressor()
regressor.fit(df_regressor, df_regressor_y)
Months_pred = regressor.predict(pred_X)

pred_X['MonthsLastRev'] = Months_pred
pred_X['MonthsLastRev'] = pred_X['MonthsLastRev'].astype(int)
pred_X

C:\Users\rodri\AppData\Local\Temp\ipykernel_21828\904101295.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_X['MonthsLastRev'] = Months_pred
C:\Users\rodri\AppData\Local\Temp\ipykernel_21828\904101295.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pred_X['MonthsLastRev'] = pred_X['MonthsLastRev'].astype(int)


,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,AcumIncidence,...,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano,MonthsLastRev
PipeId,,,,,,,,,,,,,,,,,,,,,
489780,2009,-0.111998,-0.421161,0.387956,0,False,0,-0.350055,0,0,...,0,0,0,0,1,0,0,1,0,23
489790,2012,-0.111998,-0.462561,0.216777,0,False,0,-0.350055,0,0,...,0,0,0,0,1,0,0,1,0,23
489792,2012,-0.111998,-0.362993,0.216777,1,False,0,-0.350055,0,0,...,0,0,0,0,1,0,0,1,0,23
489793,2012,-0.111998,-0.438989,0.216777,1,False,0,-0.350055,0,0,...,0,0,0,0,1,0,0,1,0,23
489981,2012,-0.111998,0.142952,0.216777,0,False,0,-0.350055,0,0,...,0,0,0,0,1,0,0,1,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
416223449,2011,-1.138929,-0.516547,7.235145,0,False,0,3.210377,0,0,...,0,0,0,0,0,0,0,1,0,29
416223471,2011,-1.138929,-0.519670,7.235145,0,False,0,3.210377,0,0,...,0,0,0,0,0,0,0,1,0,29
416425646,2011,2.385941,-0.495651,7.235145,0,False,0,3.210377,0,0,...,0,0,0,0,0,0,0,1,0,27


Restore PipeId after prediction

In [135]:
pred_X.reset_index(inplace=True)
pred_X

,PipeId,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,...,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano,MonthsLastRev
0,489780,2009,-0.111998,-0.421161,0.387956,0,False,0,-0.350055,0,...,0,0,0,0,1,0,0,1,0,23
1,489790,2012,-0.111998,-0.462561,0.216777,0,False,0,-0.350055,0,...,0,0,0,0,1,0,0,1,0,23
2,489792,2012,-0.111998,-0.362993,0.216777,1,False,0,-0.350055,0,...,0,0,0,0,1,0,0,1,0,23
3,489793,2012,-0.111998,-0.438989,0.216777,1,False,0,-0.350055,0,...,0,0,0,0,1,0,0,1,0,23
4,489981,2012,-0.111998,0.142952,0.216777,0,False,0,-0.350055,0,...,0,0,0,0,1,0,0,1,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,416223449,2011,-1.138929,-0.516547,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,1,0,29
909729,416223471,2011,-1.138929,-0.519670,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,1,0,29
909730,416425646,2011,2.385941,-0.495651,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,1,0,27
909731,416425691,2011,2.385941,-0.482059,7.235145,0,False,0,3.210377,0,...,0,0,0,0,0,0,0,1,0,25


With this MonthLastRev we estimate the inspection date

In [136]:
pred_X = pred_X.merge(last_rev_df[['PipeId', 'InspectionDate']], on = 'PipeId', how = 'left')

new_dates = []

for ind in pred_X.index:
    new_date = pred_X['InspectionDate'][ind]+pd.DateOffset(months=pred_X['MonthsLastRev'][ind])
    new_dates.append(new_date)

pred_X['InspectionDate'] = new_dates
pred_X['InspectionDate'] = pred_X['InspectionDate'].replace(pd.NaT, pd.to_datetime('2022-12-31'))

Age of the pipe

In [137]:
age_list = []

for ind in pred_X.index:
    age = int(pred_X['InspectionDate'][ind].year - pred_X['YearBuilt'][ind])
    age_list.append(age)

pred_X['Age'] = age_list

Inspection Year

In [138]:
pred_X['InspectionYear'] = pred_X['InspectionDate'].dt.year

Include Earthquake Data once the dates are known

In [140]:
# Adding Province temporal column
pred_X = pd.merge(pred_X.sort_values(by='PipeId'),provinces.sort_values(by='PipeId'),how='left',on='PipeId')
pred_X = pd.merge_asof(pred_X.sort_values(by='InspectionDate'),df_seism.sort_values(by='Fecha')
            ,left_on=['InspectionDate'],right_on=['Fecha'],by='Province')

# Dropping the extra variables
pred_X = pred_X.drop(['Fecha','Year','Seism'],axis=1)
pred_X = pred_X.sort_values(['PipeId','InspectionDate'])

# Change them to Integer
# Fill the empty values with the ones from the previous inspection
pred_X = pred_X.fillna(method='ffill')
pred_X['AcumSeism'] = pred_X['AcumSeism'].astype(int)
pred_X

,PipeId,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,PrevSeverity,...,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano,MonthsLastRev,InspectionDate,Age,InspectionYear,Province,AcumSeism
839900,489780,2009,-0.111998,-0.421161,0.387956,0,False,0,-0.350055,0,...,0,0,1,0,23,2022-03-18,13,2022,Toledo,74
839579,489790,2012,-0.111998,-0.462561,0.216777,0,False,0,-0.350055,0,...,0,0,1,0,23,2022-03-18,10,2022,Toledo,74
839576,489792,2012,-0.111998,-0.362993,0.216777,1,False,0,-0.350055,0,...,0,0,1,0,23,2022-03-18,10,2022,Toledo,74
839578,489793,2012,-0.111998,-0.438989,0.216777,1,False,0,-0.350055,0,...,0,0,1,0,23,2022-03-18,10,2022,Toledo,74
839610,489981,2012,-0.111998,0.142952,0.216777,0,False,0,-0.350055,0,...,0,0,1,0,23,2022-03-18,10,2022,Toledo,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39432,451195430,2012,0.755342,1.254764,-0.442265,7,False,0,-0.350055,0,...,0,0,1,0,23,2020-12-03,8,2020,Málaga,1504
38786,451195580,1950,0.755342,0.059637,-0.442265,0,False,0,-0.350055,0,...,0,0,1,0,23,2020-12-02,70,2020,Málaga,1504
814101,451995254,2010,-0.458934,-0.521517,-0.442265,0,False,0,-0.350055,0,...,0,0,1,0,28,2022-02-24,12,2022,Lleida,2607
602344,451995260,2003,-0.458934,0.390792,-0.442265,2,False,0,-0.350055,0,...,0,0,1,0,23,2021-09-23,18,2021,Lleida,2333


Indexing with non used columns

In [141]:
df = df.set_index(['PipeId', 'InspectionDate','Severity'])
pred_X = pred_X.set_index(['PipeId', 'InspectionDate'])


Balance training dataframe

In [142]:
# First we check how balanced is the dataset
Incidence = df[df.Incidence == 1]
not_Incidence = df[df.Incidence == 0]

# index property contains the indexes of the not_Incidence dataset
not_Incidence_indexes = not_Incidence.index

# get as many not_incidence indexes (randomly) as incidence occurrences
np.random.seed(42)
random_not_Incidence_indexes = np.random.choice(not_Incidence_indexes, len(Incidence))

# Keep those entries in not_Incidence
not_Incidence = df.loc[random_not_Incidence_indexes]

# We will form the balanced dataset concatenating Incidence and not_Incidence
balanced_df = Incidence.append(not_Incidence)
balanced_df.sort_index()

C:\Users\rodri\AppData\Local\Temp\ipykernel_21828\662246051.py:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  balanced_df = Incidence.append(not_Incidence)


,,,InspectionYear,MonthsLastRev,Incidence,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,...,Material_FG,Material_FI,Material_FO,Material_PA,Material_PE,Material_PN,Material_PV,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,InspectionDate,Severity,,,,,,,,,,,,,,,,,,,,,
489981,2016-04-28,0,2016,22,0,2012,-0.166884,0.173283,0.166398,0,False,0,...,0,0,0,0,1,0,0,0,1,0
490620,2020-05-18,0,2020,24,0,2012,0.707025,-0.412112,1.079290,0,False,0,...,0,0,0,0,1,0,0,0,1,0
490635,2017-08-31,0,2017,24,0,2012,0.707025,-0.500785,-0.381338,0,False,0,...,0,0,0,0,1,0,0,0,1,0
497136,2019-01-25,1,2019,24,1,2008,0.707025,3.917794,-0.419375,11,False,0,...,0,0,0,0,1,0,0,0,1,0
497253,2017-01-10,0,2017,24,0,2005,-0.166884,0.481415,-0.419375,1,False,0,...,0,0,0,0,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450861286,2020-03-12,0,2020,24,0,2012,0.707025,-0.483819,-0.419375,0,False,0,...,0,0,0,0,1,0,0,0,1,0
450872858,2011-12-21,1,2011,21,1,1991,0.574190,0.437210,0.166398,1,True,0,...,0,0,0,0,0,0,0,0,1,0
450873154,2012-03-05,1,2012,2,1,2012,-1.201592,-0.487603,0.166398,1,True,0,...,0,0,0,0,0,0,0,0,1,0


split train X and Y

In [143]:
train_X = balanced_df.drop(["Incidence"], axis = 1)
train_Y = balanced_df["Incidence"]

train_X = train_X.drop(["Material_FO", "Material_PV"], axis = 1)

In [144]:
train_X

,,,InspectionYear,MonthsLastRev,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,...,Material_CU,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,InspectionDate,Severity,,,,,,,,,,,,,,,,,,,,,
497136,2019-01-25,1,2019,24,2008,0.707025,3.917794,-0.419375,11,False,0,-0.354140,...,0,0,0,0,0,1,0,0,1,0
510311,2012-06-21,1,2012,17,2011,-1.390357,0.790643,-0.419375,6,False,0,-0.354140,...,0,0,0,0,0,1,0,0,1,0
586278,2012-05-01,3,2012,19,1970,-1.201592,-0.157565,0.166398,0,True,0,3.147804,...,0,0,0,0,0,0,0,0,1,0
634981,2011-02-16,1,2011,21,1999,-1.201592,0.554661,1.992183,0,False,0,3.147804,...,0,0,0,0,0,0,0,0,1,0
644491,2013-07-30,1,2013,24,1989,-0.166884,0.040765,-0.438394,2,False,0,-0.354140,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133067019,2016-04-08,0,2016,49,2001,1.406152,-0.492524,-0.419375,1,False,0,-0.354140,...,0,0,0,0,0,1,0,0,1,0
423217978,2015-09-24,0,2015,23,2011,-0.166884,-0.492842,-0.419375,0,False,0,-0.354140,...,0,0,0,0,0,1,0,0,1,0
300463684,2011-01-28,0,2011,24,2006,1.406152,0.498410,-0.419375,0,False,0,-0.354140,...,0,0,0,0,0,1,0,0,1,0


Reorder prediction columns

In [145]:
pred_X = pred_X[train_X.columns]
pred_X

,,InspectionYear,MonthsLastRev,YearBuilt,Diameter,Length,Pressure,NumConnections,BoolBridle,Corrosive,Youngs_Mod,...,Material_CU,Material_FD,Material_FG,Material_FI,Material_PA,Material_PE,Material_PN,Material_ZD,GasType_Gas natural,GasType_Gas propano
PipeId,InspectionDate,,,,,,,,,,,,,,,,,,,,,
489780,2022-03-18,2022,23,2009,-0.111998,-0.421161,0.387956,0,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0
489790,2022-03-18,2022,23,2012,-0.111998,-0.462561,0.216777,0,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0
489792,2022-03-18,2022,23,2012,-0.111998,-0.362993,0.216777,1,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0
489793,2022-03-18,2022,23,2012,-0.111998,-0.438989,0.216777,1,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0
489981,2022-03-18,2022,23,2012,-0.111998,0.142952,0.216777,0,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451195430,2020-12-03,2020,23,2012,0.755342,1.254764,-0.442265,7,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0
451195580,2020-12-02,2020,23,1950,0.755342,0.059637,-0.442265,0,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0
451995254,2022-02-24,2022,28,2010,-0.458934,-0.521517,-0.442265,0,False,0,-0.350055,...,0,0,0,0,0,1,0,0,1,0


## Incidence prediction

In [146]:
# Random Forest model
forecaster2 = RandomForestClassifier(criterion= 'entropy',
                                     max_depth= 20,
                                     max_features= 'log2',
                                     min_samples_leaf= 1,
                                     min_samples_split= 4,
                                     n_estimators= 120)

# forecaster2 = GradientBoostingClassifier(learning_rate= 0.1,
#                                            n_estimators= 60,
#                                            criterion= 'friedman_mse',
#                                            max_depth= 5,
#                                            min_samples_leaf= 1,
#                                            min_samples_split= 3,
#                                            max_features= 'sqrt')

forecaster2.fit(train_X, train_Y)
# Incidence_pred2 = forecaster2.predict(pred_X)
Incidence_pred2 = forecaster2.predict_proba(pred_X)
Incidence_pred2 = Incidence_pred2[:,1]


In [147]:
Incidence_pred2

array([0.16856551, 0.22163274, 0.14572042, ..., 0.13540795, 0.41241614,
       0.24604992])

## Export

In [148]:
pred_X.reset_index(inplace=True)

In [149]:
export = pd.DataFrame()
export['PipeId'] = pred_X["PipeId"]
export['Incidence'] = Incidence_pred2

# set indexes again to be able to re-run the model straight away
pred_X = pred_X.set_index(['PipeId', 'InspectionDate'])

export

,PipeId,Incidence
0,489780,0.168566
1,489790,0.221633
2,489792,0.145720
3,489793,0.155834
4,489981,0.223297
...,...,...
909728,451195430,0.539733
909729,451195580,0.238972
909730,451995254,0.135408
909731,451995260,0.412416


In [150]:
print(len(export[export.Incidence == 1]))
print(len(export[export.Incidence == 0]))

7
0


In [151]:
submission_set = pipes.merge(export[['PipeId', 'Incidence']], on = 'PipeId', how = 'left')
submission_set

,PipeId,Incidence
0,446859944,0.260719
1,428124500,0.425632
2,438428871,0.126523
3,429034569,0.528072
4,411184477,0.303199
...,...,...
909728,235426649,0.109661
909729,235426673,0.204392
909730,235426707,0.195532
909731,190956601,0.157978


In [152]:
submission_set.to_csv('Asan-Dopollos.csv', index=False)